# TM10007 Assignment template -- ECG data

### Importing modules

In [38]:

!pip install sklearn numpy matplotlib imbalanced-learn statsmodels

import os
import zipfile
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import statsmodels
import seaborn
import warnings

from scipy.stats import shapiro, lognorm
from scipy.spatial.distance import cdist
from sklearn.model_selection import StratifiedShuffleSplit, learning_curve, GridSearchCV, StratifiedKFold, cross_val_score, KFold, train_test_split, RandomizedSearchCV #, multipletests
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn import datasets as ds, model_selection, metrics, neighbors
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from statsmodels.stats.multitest import multipletests

# Classifiers
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, SelectFromModel
from sklearn.decomposition import PCA
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

# Required and still not used:
#import torch
#import seaborn

## Preprocessing

### Importing our data

In [29]:
cwd = os.getcwd() # This fn will return the Current Working Directory

zip_path = os.path.join(cwd, 'ecg', 'ecg_data.zip')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(cwd, 'ecg'))

data_path = os.path.join(cwd, 'ecg', 'ecg_data.csv')
data = pd.read_csv(data_path, index_col=0)

### Exploring our data

In [30]:
# split labels from data
X = data.loc[:, data.columns != 'label']  #alles behalve label
y = data['label']  # labels

# normal / abnormal ECGs
total_abnormal_ECG = np.count_nonzero(y) 
total_normal_ECG = y.size -np.count_nonzero(y) 
percentage_abnormal = total_abnormal_ECG / (total_abnormal_ECG + total_normal_ECG)*100

print(f'{total_abnormal_ECG} people have an abnormal ECG')
print(f'{total_normal_ECG} people have a normal ECG')
print(f'The percentage of abnoraml ECGs in this dataset is {percentage_abnormal} %')

146 people have an abnormal ECG
681 people have a normal ECG
The percentage of abnoraml ECGs in this dataset is 17.654171704957676 %


### Testing for normal distribution - before outliers and missing data correction

In [31]:
# # Normally distributed
# stat = []
# p = []
# for col in X.columns:
#     if X[col].dtype == 'float64' or X[col].dtype == 'int64':
#         s, pv = shapiro(X[col])
#         stat.append(s)
#         p.append(pv)
#     else:
#         stat.append(None)
#         p.append(None)

# # hier nog iets printen of het wel/niet normally distributed is?

### Missing data
- Removing features if there is lot of data missing (replace all for a value)
- Removing samples (in this case patients) if there is a lot of data missing
- Imputation for generating data to fill us missing values -> median

In [32]:
# # Missing data
# X = X.replace(0, np.nan)  # make all zeros to NaN
# nan_count = X.isna().sum().sum()  # count missing data -> 10500 in our dataset

# # Delete missing data when > --% of feature of sample is missing
# X = X.dropna(axis='columns', how='all') # deletes a feature if all values of a column (so feature) are empty
# X = X.dropna(axis='rows', how='all') # deletes a patient if all values of a row (so sample) are empty

# # Missing data to median per feature
# for column in X.columns:
#     X[column].fillna(X[column].median(), inplace=True)

### Outliers
- Detect outliers using Z-score since data is not nornally distributed
- Replace outliers by the median of that feature
- Print -> check wether the outliers are changed

In [33]:
# # supress performance warning
# warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# # create a new dataframe to store the results
# results = pd.DataFrame({'Column': X.columns, 'W': stat, 'p-value': p}) 
# mean_p_value = results['p-value'].mean()  # p-value is really small. If p-value is bigger than 0.05, then data is normally distributed. SO its not
# median_p_value = results['p-value'].median()  # p-value is really small. If p-value is bigger than 0.05, then data is normally distributed. SO its not

# # Outliers: Tukey's fence 
# k=3
# fences=pd.DataFrame()
# outliers = pd.DataFrame(False, index=X.index, columns=X.columns) # create an empty DataFrame for outliers

# for col in X.columns:
#     q1, q3 = np.percentile(X[col], [25, 75])
#     iqr = q3 - q1
#     lower_fence = q1 - k*iqr
#     upper_fence = q3 + k*iqr
#     fences[col]=[lower_fence, upper_fence]
#     for row in X.index:
#         if X.loc[row, col] < lower_fence or X.loc[row, col] > upper_fence:
#             outliers.loc[row, col] = True # mark the place as an outlier

# row_count = (outliers == True).sum(axis=1)
# col_count = (outliers == True).sum(axis=0)
# total_count = row_count.sum() + col_count.sum()
# print(f'The total number of outliers in dataset x is {total_count}')

# # create a copy of x to modify
# new_x = X.copy()

# # replace outliers with median of x by column
# for col in outliers.columns:
#     median = X.loc[outliers[col] == False, col].median() # median of column where outlier is False
#     new_x.loc[outliers[col], col] = median # replace outliers with median

### Testing for normal distribution - after outliers and missing data correction

### Removing zero Variance features
not sure yet where to put that

### Putting it all together in pipelines

### Pipeline 3
- RobustScaler --> PCA + univariate --> Gaussian Naive Bayes

In [40]:
# Define outer and inner cross validation
outer_cv = StratifiedKFold(n_splits=4)
inner_cv = StratifiedKFold(n_splits=2) #, shuffle=True, random_state=42)

for train_index, test_index in outer_cv.split(X, y): 
    X_train = X.transpose()[train_index]
    X_train = X_train.transpose()
    print(X_train.shape)
    # print size of X_train
    y_train = y[train_index]
    
    X_test = X.transpose()[test_index]
    X_test = X_test.transpose()
    y_test = y[test_index]
    # print size of X_test
    print(X_test.shape)

    # balance the classes, so training set consists of 50% normal and 50% abnormal ECG's
    ros = RandomOverSampler(sampling_strategy='minority')
    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
    X_train = X_resampled
    y_train = y_resampled   

    # pipeline 1:
    # pipeline 2: 
    ## PIPELINE 3: RobustScaler --> PCA + univariate --> Gaussian Naive Bayes
    # Define pipeline 3
    pipeline_3 = Pipeline([
        ('scaler', RobustScaler()),
        ('var_threshold', VarianceThreshold(threshold=0.0)),
        ('pca', PCA()),
        ('univar_feat_sel', SelectKBest(f_classif)),
        ('clf', GaussianNB())
    ])
    # Define hyperparameters of pipeline 3
    param_grid = {
    'pca__n_components': [0.5,0.75,0.9],
    'univar_feat_sel__k': [5, 10, 20, 30],
    'clf__var_smoothing': np.logspace(0,-9, num=100),
    }

    # Perform grid search with inner cross-validation, part 1
    rand_search = RandomizedSearchCV(pipeline_3, param_distributions=param_grid, n_iter=50, cv=inner_cv, scoring='roc_auc') #klopt roc_auc?
    rand_search.fit(X_train, y_train)

    # Print the best hyperparameters and score
    print('Best hyperparameters after first grid search:', rand_search.best_params_)
    print('Best score after first grid search:', rand_search.best_score_)

    # Apply PCA
    pca = PCA(n_components = rand_search.best_params_['pca__n_components'])
    X_train = pca.fit_transform(X_train)
    print(f'Size of X_Train after PC {X_train.shape}')

    # Get the selected feature indices from the univariate feature selection
    sel_kb = SelectKBest(f_classif, k='all')
    sel_kb.fit(X_train, y_train)
    p_values = sel_kb.pvalues_

    reject_fdr, pvals_fdr, _, _ = multipletests(pvals=p_values, alpha=0.05, method='fdr_bh')
    features_selected=np.array(np.where(reject_fdr)[0])
    print(f'Size of selected features {features_selected.shape}')

    # Apply univariate feature selection on X_train
    print(f'This is the size of X_train before univariate: {X_train.shape}')
    X_train = X_train[:, features_selected]
    print(f'This is the size of X_train after univariate: {X_train.shape}')

    # Train the classifier on the selected features with the best hyperparameters to create best trained classifier
    clf_3 = GaussianNB(var_smoothing=rand_search.best_params_['clf__var_smoothing'], cv=inner_cv)
    clf_3.fit(X_train, y_train)

    y_pred = clf_3.predict(X_train)

    if hasattr(clf_3, 'predict_proba'):
    # The first column gives the probability for class = 0, so we take
    # the second which gives the probability class = 1:
        y_score = clf_3.predict_proba(X_train)[:, 1]
    else:
        y_score = y_pred

    auc=metrics.roc_auc_score(y_train, y_score)

    print(f'The auc of the training data is {auc}')
    # cv results
    #pd.DataFrame(grid_nb.cv_results_)

    # Evaluate the classifier on the test data
    X_test = pca.transform(X_test)  # Apply the PCA transformation to the test data
    X_test = X_test_pca[:, features_selected]  # Apply the feature selection to the PCA-transformed test data
    y_pred = clf_3.predict(X_test)
    auc = metrics.roc_auc_score(y_test, y_pred)
    print(f'The auc of the test data is {auc}')

    # pipeline 4: 
    # pipeline 5:
    # pipeline 6:
    # pipeline 7:



(620, 9000)
(207, 9000)


### PIPELINE 2
- RobustScaler --> PCA + univariate --> SVM_linear

In [ ]:
# Define outer and inner cross validation
outer_cv = StratifiedKFold(n_splits=4)
inner_cv = StratifiedKFold(n_splits=2) #, shuffle=True, random_state=42)

for train_index, test_index in outer_cv.split(X, y): 
    X_train = X.transpose()[train_index]
    X_train = X_train.transpose()
    print(X_train.shape)
    # print size of X_train
    y_train = y[train_index]
    
    X_test = X.transpose()[test_index]
    X_test = X_test.transpose()
    y_test = y[test_index]
    # print size of X_test
    print(X_test.shape)

    # balance the classes, so training set consists of 50% normal and 50% abnormal ECG's
    ros = RandomOverSampler(sampling_strategy='minority')
    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
    X_train = X_resampled
    y_train = y_resampled   

    ## PIPELINE 2: RobustScaler --> PCA + univariate --> SVM_linear
    # Define pipeline 2
    pipeline_2 = Pipeline([
        ('scaler', RobustScaler()),
        ('var_threshold', VarianceThreshold(threshold=0.0)),
        ('pca', PCA()),
        ('univar_feat_sel', SelectKBest(f_classif)),
        ('clf', SVM())
    ])
    # Define hyperparameters of pipeline 3
    param_grid = {
    'pca__n_components': [0.5,0.75,0.9],
    'univar_feat_sel__k': [5, 10, 20, 30],
    'classifier__C': np.logspace(-3, 1, 20)
    'classifier__kernel': ['linear']
    }

    # Perform grid search with inner cross-validation, part 1
    rand_search = RandomizedSearchCV(pipeline_2, param_grid=param_grid, cv=inner_cv, scoring='roc_auc') # optimize parameters
    rand_search.fit(X_train, y_train)

    # Print the best hyperparameters and score
    print('Best hyperparameters after first grid search:', rand_search.best_params_)
    print('Best score after first grid search:', rand_search.best_score_)

    # Apply PCA
    pca = PCA(n_components = rand_search.best_params_['pca__n_components'])
    X_train = pca.fit_transform(X_train)
    print(f'Size of X_Train after PC {X_train.shape}')

    # Get the selected feature indices from the univariate feature selection
    sel_kb = SelectKBest(f_classif, k='all')
    sel_kb.fit(X_train, y_train)
    p_values = sel_kb.pvalues_

    reject_fdr, pvals_fdr, _, _ = multipletests(pvals=p_values, alpha=0.05, method='fdr_bh')
    features_selected=np.array(np.where(reject_fdr)[0])
    print(f'Size of selected features {features_selected.shape}')

    # Apply univariate feature selection on X_train
    print(f'This is the size of X_train before univariate: {X_train.shape}')
    X_train = X_train[:, features_selected]
    print(f'This is the size of X_train after univariate: {X_train.shape}')

    # Train the classifier on the selected features with the best hyperparameters to create best trained classifier
    clf_2 = SVC(C=rand_search.best_params_['classifier__C'], kernel=rand_search.best_params_['classifier__kernel'])
    clf_2.fit(X_train, y_train)

    y_pred = clf_2.predict(X_train)

    if hasattr(clf_2, 'predict_proba'):
    # The first column gives the probability for class = 0, so we take
    # the second which gives the probability class = 1:
        y_score = clf_2.predict_proba(X_train)[:, 1]
    else:
        y_score = y_pred

    auc = metrics.roc_auc_score(y_train, y_score)

    print(f'The auc of the training data is {auc}')
    # cv results
    #pd.DataFrame(grid_nb.cv_results_)

    # Evaluate the classifier on the test data
    X_test = pca.transform(X_test)  # Apply the PCA transformation to the test data
    X_test = X_test_pca[:, features_selected]  # Apply the feature selection to the PCA-transformed test data
    y_pred = clf_2.predict(X_test)
    auc = metrics.roc_auc_score(y_test, y_pred)
    print(f'The auc of the test data is {auc}')



### PIPELINE 4

In [ ]:
# Define outer and inner cross validation
outer_cv = StratifiedKFold(n_splits=4)
inner_cv = StratifiedKFold(n_splits=10) #, shuffle=True, random_state=42)

for train_index, test_index in outer_cv.split(X, y): 
    X_train = X.transpose()[train_index]
    X_train = X_train.transpose()
    print(X_train.shape)
    # print size of X_train
    y_train = y[train_index]
    
    X_test = X.transpose()[test_index]
    X_test = X_test.transpose()
    y_test = y[test_index]
    # print size of X_test
    print(X_test.shape)

    # balance the classes, so training set consists of 50% normal and 50% abnormal ECG's
    ros = RandomOverSampler(sampling_strategy='minority')
    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
    X_train = X_resampled
    print(f'Size of X_train after resampling {X_train.shape}')
    y_train = y_resampled   

    ## PIPELINE 4: RobustScaler --> PCA + univariate --> Quadratic Discriminant
    # Define pipeline 4
    pipeline_4 = Pipeline([
        ('scaler', RobustScaler()),
        ('var_threshold', VarianceThreshold(threshold=0.0)),
        ('pca', PCA()),
        ('univar_feat_sel', SelectKBest(f_classif)),
        ('clf', QuadraticDiscriminantAnalysis())
    ])
    # Define hyperparameters of pipeline 3
    param_grid = {
    'pca__n_components': [0.5,0.75,0.9,0.95,0.99],
    'univar_feat_sel__k': ['all']
    }

    # Perform grid search with inner cross-validation, part 1
    rand_search = RandomizedSearchCV(pipeline_4, param_distributions=param_grid,n_iter = 50, cv=inner_cv, scoring='roc_auc') # optimize parameters
    rand_search.fit(X_train, y_train)

    # Print the best hyperparameters and score
    print('Best hyperparameters after first grid search:', rand_search.best_params_)
    print('Best score after first grid search:', rand_search.best_score_)

    # Apply PCA
    pca = PCA(n_components = rand_search.best_params_['pca__n_components'])
    X_train = pca.fit_transform(X_train)
    print(f'Size of X_Train after PC {X_train.shape}')

    # Get the selected feature indices from the univariate feature selection
    sel_kb = SelectKBest(f_classif, k='all')
    sel_kb.fit(X_train, y_train)
    p_values = sel_kb.pvalues_

    reject_fdr, pvals_fdr, _, _ = multipletests(pvals=p_values, alpha=0.05, method='fdr_bh')
    features_selected=np.array(np.where(reject_fdr)[0])
    print(f'Size of selected features {features_selected.shape}')
    
    # Apply univariate feature selection on X_train
    print(f'This is the size of X_train before univariate: {X_train.shape}')
    X_train = X_train[:, features_selected]
    print(f'This is the size of X_train after univariate: {X_train.shape}')

    # Train the classifier on the selected features with the best hyperparameters to create best trained classifier
    clf_4 = QuadraticDiscriminantAnalysis()
    clf_4.fit(X_train, y_train)

    y_pred = clf_4.predict(X_train)

    if hasattr(clf_4, 'predict_proba'):
    # The first column gives the probability for class = 0, so we take
    # the second which gives the probability class = 1:
        y_score = clf_4.predict_proba(X_train)[:, 1]
    else:
        y_score = y_pred

    auc = metrics.roc_auc_score(y_train, y_score)

    print(f'The auc of the training data is {auc}')
    # cv results
    #pd.DataFrame(grid_nb.cv_results_)

    # Evaluate the classifier on the test data
    X_test = pca.transform(X_test)  # Apply the PCA transformation to the test data
    X_test = X_test[:, features_selected]  # Apply the feature selection to the PCA-transformed test data
    y_pred = clf_4.predict(X_test)
    auc = metrics.roc_auc_score(y_test, y_pred)
    print(f'The auc of the test data is {auc}')


In [ ]:
# Define the pipelines
#1
def pipeline1(X_train, y_train, X_test, y_test):
    # Preprocessing
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Variance Threshold
    vt = VarianceThreshold(threshold=0.0)
    X_train = vt.fit_transform(X_train)
    X_test = vt.transform(X_test)

    # PCA
    n_samples = X_train.shape[0]
    n_features = X_train.shape[1]
    n_features = min(n_samples, n_features)
    pca = PCA(n_components=n_features)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    # Univariate selection
    kb = SelectKBest(f_classif, k='all') # juiste code erin als af
    X_train = kb.fit_transform(X_train, y_train)
    p_values_train = kb.pvalues_
    reject_fdr, pvals_fdr, _, _ = multipletests(pvals=p_values_train, alpha=0.05, method='fdr_bh')
    features_selected_train = np.array(np.where(reject_fdr)[0])
    print(features_selected_train.shape)
    X_train = X_train[:,features_selected_train]

    X_test = kb.transform(X_test)
    p_values_test = kb.pvalues_
    reject_fdr, pvals_fdr, _, _ = multipletests(pvals=p_values_test, alpha=0.05, method='fdr_bh')
    features_selected_test = np.array(np.where(reject_fdr)[0])
    print(features_selected_test.shape)
    X_test = X_test[:,features_selected_test]

    # KNN classifier
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    roc_auc_score = roc_auc_score(y_test, y_pred)

    return roc_auc_score

def pipeline2(X_train, y_train, X_test, y_test, alpha=0.5, n_neighbors=5):
    # Preprocessing
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Variance Threshold
    vt = VarianceThreshold(threshold=0.0)
    X_train = vt.fit_transform(X_train)
    X_test = vt.transform(X_test)

    # LASSO regularization
    lasso = Lasso(alpha=alpha)
    X_train = lasso.fit_transform(X_train, y_train)
    X_test = lasso.transform(X_test)

    # KNN classifier
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    roc_auc_score = roc_auc_score(y_test, y_pred)

    return roc_auc_score

# Define the parameters for grid search
param_grid1 = {
    'n_pca': [], #doesn't have parameters that need grid_search
    'k_features': [], #doesn't have parameters that need grid_search
    'n_neighbors': list(range(1, 26, 2))
}

param_grid2 = {
    'alpha': [np.logspace(-5, 1, 100)],
    'n_neighbors': list(range(1, 26, 2))
}

# Define the outer and inner cross-validation
outer_cv = StratifiedKFold(n_splits=4) #als we 4 splits doen, wordt de data in 25%/75% gesplit
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Perform the grid search using nested cross-validation
best_scores1 = []
best_scores2 = []

best_n_neighbors = []
results_1 = []
results_2 = []

for train_index, test_index in outer_cv.split(X, y): 
    X_train = X.transpose()[train_index]
    X_train = X_train.transpose()
    print(X_train.shape)
    y_train = y[train_index]
    
    X_test = X.transpose()[test_index]
    X_test = X_test.transpose()
    y_test = y[test_index]
    print(X_test.shape)

    # Create a grid search to find the optimal k using a gridsearch and 10-fold cross validation
#     grid_search1 = GridSearchCV(estimator=pipeline1, param_grid=param_grid1, cv=inner_cv, scoring='roc_auc') #error logisch, want pipeline1 is een functie
#     grid_search1.fit(X_train, y_train)
#     best_scores1.append(grid_search1.best_score_)

#     grid_search2 = GridSearchCV(estimator=pipeline2, param_grid=param_grid2, cv=inner_cv, scoring='roc_auc')
#     grid_search2.fit(X_train, y_train)
#     best_scores2.append(grid_search2.best_score_)

# # Print the best scores and the average score for each pipeline
# print(best_scores1)
# print(best_scores2)

# # Get resulting classifier, pipeline 1
# best_1 = grid_search1.best_estimator_
# print(f'Best classifier: k={best_1.n_neighbors}')
# best_n_neighbors.append(best_1.n_neighbors)

# # Test the classifier on the test data, pipeline 1
# probabilities = best_1.predict_proba(X_test)
# scores_1 = probabilities[:, 1]

# # Get the auc, pipeline 1
# auc_1 = metrics.roc_auc_score(y_test, scores_1)
# results_1.append({
#     'auc': auc_1,
#     'k': best_1.n_neighbors,
#     'set': 'test'
# })

# # Create results dataframe and plot it, pipeline 1
# results = pd.DataFrame(results_1)
# seaborn.boxplot(y='auc', x='set', data=results_1)

# optimal_n = int(np.median(best_n_neighbors))
# print(f"The optimal N={optimal_n}")

(620, 9000)
(207, 9000)
(620, 9000)
(207, 9000)
(620, 9000)
(207, 9000)
(621, 9000)
(206, 9000)


In [ ]:
print(X_test)